In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib as plt
import math
import random
import subprocess
import openpyxl
import re
import win32com.client  

from io import StringIO
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.visualization.scatter import Scatter


In [ ]:
def extract_subcatchments(file_path):
   
    with open(file_path, 'r') as file:
        content = file.readlines()

    in_subcatchments_section = False
    subcatchments_data = []

    for line in content:
        line = line.strip()

        # Check if we're in the [SUBCATCHMENTS] section
        if line.startswith("[SUBCATCHMENTS]"):
            in_subcatchments_section = True
            continue

        # Exit the section if a new section starts
        if in_subcatchments_section and line.startswith("[") and line.endswith("]"):
            break

        # Parse data in the [SUBCATCHMENTS] section
        if in_subcatchments_section and line and not line.startswith(";"):
            parts = line.split()
            if len(parts) >= 3:
                subcatchment_name = parts[0]
                area = parts[3]  
                Imp0 = parts[4]  
                width = parts[5] 
                subcatchments_data.append((subcatchment_name, area, Imp0))

    return subcatchments_data


In [ ]:
swmmworking_directory= 'SWMMDirectory'
os.chdir(swmmworking_directory)

# File path to the input file
file_path = "SWMMModel.inp"  # Replace with the actual path to your file

# Extract subcatchment data
subcatchments_data = extract_subcatchments(file_path)

# Create a DataFrame and save it for further analysis
df_subcatchments = pd.DataFrame(subcatchments_data, columns=["Name", "Area", "%Imperv"])
print(df_subcatchments)

# Optionally, save to a CSV file
output_csv = "subcatchments.csv"
df_subcatchments.to_csv(output_csv, index=False)
# print(f"Subcatchments data saved to {output_csv}")

In [ ]:
def LIDCombo(list):
    # Define the allowed LIDs for each subcatchment
    allowed_lids = {
        1: ["BR", "PP", "IT", "RB", "GR", "VS"],
        2: ["BR", "PP", "IT", "RB", "GR", "VS"],
        3: ["BR", "PP", "IT", "RB", "GR", "VS"],
        4: ["BR", "PP", "IT", "RB", "GR", "VS"],
        5: ["BR", "PP", "IT", "RB", "GR", "VS"],
        6: ["BR", "PP", "IT", "RB", "GR", "VS"],
        7: ["BR", "PP", "IT", "RB", "GR", "VS"],
        8: ["BR", "PP", "IT", "RB", "GR", "VS"],
        9: ["BR", "PP", "IT", "RB", "GR", "VS"],
        10: ["BR", "PP", "IT", "RB", "GR", "VS"],
        11: ["BR", "PP", "IT", "RB", "GR", "VS"],
        12: ["BR", "PP", "IT", "RB", "GR", "VS"],
        13: ["BR", "PP", "IT", "RB", "GR", "VS"],,
        14: ["BR", "PP", "IT", "RB", "GR", "VS"],
        15: ["BR", "PP", "IT", "RB", "GR", "VS"],
        16: ["BR", "PP", "IT", "RB", "GR", "VS"],
        17: ["BR", "PP", "IT", "RB", "GR", "VS"],
        
    }
    
    # Extract subcatchments from the dictionary keys
    selected_subcatchments = allowed_lids.keys()
    
    # Construct key pairs only for allowed LIDs
    keys = [(str(subcatchment), lid) for subcatchment in selected_subcatchments for lid in allowed_lids[subcatchment]]
    
    # Ensure the list has the right number of values
    if len(list) != len(keys):
        raise ValueError("Input list length does not match the required LID assignment size.")
    
    # Create the LID Plan dictionary
    LIDPlan = {keys[i]: list[i] for i in range(len(keys))}
    
    return LIDPlan


In [ ]:
def update_LID(new_values):

    file_path = "SWMMModel.inp"  # Replace with your input file path
    output_file_path = "SWMMUpdate.inp"  # Replace with your desired output file path
   
    with open(file_path, 'r') as file:
        content = file.readlines()

    in_LID_USAGE_section = False
    updated_content = []
    subcatchment_LID = {}

    # Calculate the sum of LID areas for each subcatchment
    for (subcatchment, _), value in new_values.items():
        if subcatchment not in subcatchment_LID:
            subcatchment_LID[subcatchment] = 0
        subcatchment_LID[subcatchment] += value
        
    # print(subcatchment_LID)
    for line in content:
        stripped_line = line.strip()

        # Check if we're in the [LID_USAGE] section
        if stripped_line.startswith("[LID_USAGE]"):
            in_LID_USAGE_section = True
            updated_content.append(line)
            continue

        # Exit the section if a new section starts
        if in_LID_USAGE_section and stripped_line.startswith("[") and stripped_line.endswith("]"):
            in_LID_USAGE_section = False
        
        # Update the LID area in the section
        if in_LID_USAGE_section and stripped_line and not stripped_line.startswith(";"):
            parts = stripped_line.split()
            if len(parts) >= 5:  # Ensure the line has enough columns
                subcatchment_name = parts[0]
                LID_name = parts[1]
                key = (subcatchment_name, LID_name)

                # Update LID Number
                if key in new_values:
                    subcatchment_area = next(
                        (float(item[1]) for item in subcatchments_data if item[0] == subcatchment_name),
                        None
                    )
                    if subcatchment_area is not None:
                        parts[2] = str(int(new_values[key] * subcatchment_area * 0.43560) )  # Update LID Number LID Area/1000

                # # Update LID Width for specific LID names
                # if LID_name in {"GR", "PP", "RD"}:
                #     subcatchment_width = next(
                #         (float(item[3]) for item in subcatchments_data if item[0] == subcatchment_name),
                #         None
                #     )
                #     if subcatchment_width is not None:
                #         parts[4] = str(new_values.get(key, 1) * subcatchment_width)  # Scale the width by a factor

                line = " ".join(parts) + "\n"  # Reconstruct the line

        updated_content.append(line)

    # Update %Imperv values for subcatchments
    for i, line in enumerate(updated_content):
        stripped_line = line.strip()
        if stripped_line.startswith("[SUBCATCHMENTS]"):
            for j in range(i + 1, len(updated_content)):
                subcatchment_line = updated_content[j].strip()
                if subcatchment_line.startswith("[") and subcatchment_line.endswith("]"):
                    break
                if subcatchment_line and not subcatchment_line.startswith(";"):
                    parts = subcatchment_line.split()
                    if len(parts) >= 4:
                        subcatchment_name = parts[0]

                        impold = next(
                            (float(item[2]) for item in subcatchments_data if item[0] == subcatchment_name),
                            None
                        )
                        a = subcatchment_LID.get(subcatchment_name, 0) / 100 # Sum of LID areas for the subcatchment

                        # print(a,subcatchment_name)
                        if impold is not None and a < 1:  # Ensure no division by zero
                            impnew = (impold - a*100) / (1 - a)
                            if impnew < 1:
                                impnew = 1
                            parts[4] = f"{impnew:.2f}"  # Update %Imperv with new value
                            updated_content[j] = " ".join(parts) + "\n"
                            # print(a, impold, impnew)

    # Save the updated content to a new file
    with open(output_file_path, 'w') as file:
        file.writelines(updated_content)

    # print(f"File updated and saved to {output_file_path}")


In [ ]:
def SWMM():
    
    input_swmmfile = "SWMMUpdate.inp"
    report_swmmfile = "SWMMUpdate.rpt"
    ## Initial Coefficients 
    conversion = 0.133681   # Convert from gallon to ft3
    
    #SWMM
    swmm_executable = "runswmm"  # Path to SWMM executable
   
    try:
        
# Run SWMM MODEL
        # Step 1: Run SWMM simulation
        command = [swmm_executable, input_swmmfile, report_swmmfile]
        subprocess.run(command, check=True, capture_output=True, text=True)
        # print("SWMM simulation completed successfully!")

# Reading Total Surface Runoff
        # Step 2: Extract total flow volume at outfall
        
        file_path = report_swmmfile
        section_found = False
        total_flow = None
        
        with open(file_path, 'r') as file:
            for line in file:
                if "Outfall Node" in line:
                    section_found = True
                    # print("Found 'Outfall Node' section!")  # Debugging
                    continue


                if section_found:
                    if line.strip().startswith("OF1"):  # Look for the line starting with "OF1"
                        columns = line.split()
                        if len(columns) >= 4:
                            max_flow = float(columns[3])    # Extract the Max Flow value
                            total_flow = float(columns[4])  # Extract the Total Volume Flow value
                        break

                    if line.strip() == "":  # Stop searching if an empty line is found
                        section_found = False

        if total_flow is None:
            raise ValueError("OF1 Total Flow value not found in the report.")
        
        # print(f"Total Volume Inflow (cfs): {total_flow}")
       
        total_flow_cf = total_flow * conversion * 1000000
        
        return total_flow_cf , max_flow

    except Exception as e:
        print(f"An error occurred: {e}")
        return None  # Return None if there's an error

In [ ]:
def UrbanFlow (values_list):

    swmmworking_directory= 'SWATDirectory'
    os.chdir(swmmworking_directory)
    
    LIDPlan = LIDCombo(values_list)           # Convert binary list to LID combination
    
    update_LID(LIDPlan)                       # Update the SWMM model with LID selection
    
    totalflow,maxflow = SWMM()                # Run SWMM and get total flow
    
    return totalflow, maxflow   

In [ ]:
def Update_BMPs(Dimensions):
    
    # ----- Update res_dat.swf -----
    input_file = "res_dat.swf"
    output_file = "res_dat.swf"
    
    # Read all lines from the file
    with open(input_file, "r") as file:
        lines = file.readlines()

    # Define the target row (row 5, index 4) and the last four columns
    target_rows = [4, 5, 6]
    target_cols = [-4, -3, -2, -1]


    # Loop over the target rows and update the last four values.
    for i, row_index in enumerate(target_rows):
        if row_index < len(lines):
            parts = re.split(r'\s+', lines[row_index].strip())

            # Update the target columns using multipliers 1, 10, 2, and 20 respectively.
            parts[target_cols[0]] = str(Dimensions[i])
            parts[target_cols[1]] = str(Dimensions[i] * Dimensions[i+3])    # +n if there is n ponds
            parts[target_cols[2]] = str(1 * Dimensions[i])
            parts[target_cols[3]] = str(1 * Dimensions[i] * Dimensions[i+3])   # +n if there is n ponds
            # Reconstruct the modified line
            lines[row_index] = " ".join(parts) + "\n"

    # Write the modified lines back to the file
    with open(output_file, "w") as file:
        file.writelines(lines)
        
    # print(f"Modified file")

    # ----- Update hydrology.res -----
    input_file2 = "hydrology.res"
    output_file2 = "hydrology.res"
    
    with open(input_file2, "r") as file:
        lines2 = file.readlines()

    # Define the target row (row 3, index 2) and columns 4,5,6,7 (indices 3, 4, 5, 6)
    target_rows2 = [2, 3, 4]
    target_cols2 = [3, 4, 5, 6]

    # 4 Ponds
    # target_rows2 = [2, 3, 4, 5]
    # target_cols2 = [3, 4, 5, 6]
    # Loop over the target rows and update the specified columns.
    for i, row_index in enumerate(target_rows2):
        if row_index < len(lines2):
            parts = re.split(r'\s+', lines2[row_index].strip())
            parts[target_cols2[0]] = str(Dimensions[i])
            parts[target_cols2[1]] = str(Dimensions[i] * Dimensions[i+3])
            parts[target_cols2[2]] = str(1 * Dimensions[i])
            parts[target_cols2[3]] = str(1 * Dimensions[i] * Dimensions[i+3])
            lines2[row_index] = " ".join(parts) + "\n"

    with open(output_file2, "w") as file:
        file.writelines(lines2)
        
    # print(f"Modified file")


In [ ]:
def SWAT (swatworking_directory, UpsIn):

    coeff = 0.0283168                           #     Converting form ft3 to m3
    unit_filter = 278                           #     Downstream river
    
    # swatworking_directory = 'TxtInOut'
    os.chdir(swatworking_directory)
    swatworking_directory
    input_swatfile_path = 'exco_om.exc'   
    output_swatfile_path = 'exco_om.exc'  
    swat_executable = "rev61.0_64rel.exe"

    try:

# Modification of upstream inlet 
        # Step 1: Modify a specific file    
        target_column = 'flo'                        
         
        with open(input_swatfile_path, 'r') as file:
            file_lines = file.readlines()

        header_line_index = 1
        data_start_index = 2
        header = file_lines[header_line_index].strip().split()
        data_lines = file_lines[data_start_index:]

        data_str = "\n".join(data_lines)
        data = pd.read_csv(StringIO(data_str), delim_whitespace=True, header=None, names=header)

        if target_column in data.columns:
            data[target_column] = UpsIn * coeff
        else:
            raise ValueError(f"Column '{target_column}' not found in the file.")

        with open(output_swatfile_path, 'w') as file:
            file.writelines(file_lines[:data_start_index])
            data.to_csv(file, sep='\t', index=False, header=False)

        # print(f"Modified file saved")

# Running SWAT+
        # Step 2: Run SWAT+ Simulation
        
        subprocess.run([swat_executable], check=True, capture_output=True, text=True)
        

# Read SWAT+ Output
   
        initial_file_path = "channel_sd_day.txt"
        intermediate_csv_path = "channel_sd_day.csv"

        data = pd.read_csv(initial_file_path, delim_whitespace=True, skiprows=1)
        data.to_csv(intermediate_csv_path, index=False)

        columns_to_keep = ['unit', 'flo_out']
        selected_columns_path = "columns.csv"
        selected_data = data[columns_to_keep]
        selected_data.to_csv(selected_columns_path, index=False)

        rows_to_remove = [0]
        modified_columns_path = "columns1.csv"
        data_without_rows = selected_data.drop(index=rows_to_remove)
        data_without_rows.to_csv(modified_columns_path, index=False)

        filtered_file_path = "flo_out.csv"
        data_clean = pd.read_csv(modified_columns_path)
        filtered_data = data_clean[data_clean['unit'] == unit_filter]
        filtered_data.to_csv(filtered_file_path, index=False)

# Peak Outflow
        # Step 6: Extract outflow
        
        data2 = pd.read_csv('flo_out.csv')
        outflow = max(data2['flo_out'])
        # print(f"Outflow: {outflow}")
    
        return outflow

    except subprocess.CalledProcessError as e:
        print(f"Error during subprocess execution: {e.stderr}")
    except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:
def RuralFlow (Dimensions, totalflow):

    swatworking_directory= 'SWATDirectory'
    os.chdir(swatworking_directory)

    Update_BMPs (Dimensions)
    outflow = SWAT(swatworking_directory, totalflow)                 # Run SWMM and get total flow
   
    
    return outflow 
def TotalFlow (values_list):

    LIDList =  values_list[:72]
 
    PondsDimensions = values_list[72:78]

    urbantotalflow, maxurbanflow = UrbanFlow (LIDList)
  
    ruralmaxflow = RuralFlow (PondsDimensions, urbantotalflow)
    
    return (maxurbanflow, ruralmaxflow)
    
def LIDAare(new_values, subcatchments_data):
    
    lid_areas = {}

    for (subcatchment_name, lid_name), value in new_values.items():
        # Find the area of the subcatchment
        subcatchment_area = next(
            (float(item[1]) for item in subcatchments_data if item[0] == subcatchment_name),
            None
        )
        
        if subcatchment_area is not None:
            # Calculate the LID's area
            
            lid_area = int(value * subcatchment_area * 0.01) 
            if lid_name not in lid_areas:
                lid_areas[lid_name] = 0
            lid_areas[lid_name] += lid_area
        else:
            print(f"Warning: Subcatchment '{subcatchment_name}' not found in subcatchments data.")
    # print (lid_areas) 

    return lid_areas
def LIDCost(lid_areas):
    LocFactor = 0.907
    ENRCCI = 1.24

    # Cost formulas for each LID type
    cost_formulas = {
        "BR": lambda x: 3.33 * x * 3.446 * 80495,
        "IT": lambda x: 2    * x * 3.446 * 80770,
        "PP": lambda x: 1    * x * 3.446 * 283347,
        "RB": lambda x: 2.5  * x * 3.446 * 163957,
        "VS": lambda x: 1    * x * 3.446 * 38207,
        "GR": lambda x: 0.5  * x * 3.446 * 283347 * 1.16,
    }

    # Calculate the total cost
    total_cost = 0
    for lid, area in lid_areas.items():
        if area == 0:
            cost = 0
        elif lid in cost_formulas:
            cost = cost_formulas[lid](area)
        else:
            print(f"Warning: No cost formula defined for LID type '{lid}'.")
            cost = 0
        total_cost += cost

    total_cost = total_cost * LocFactor * ENRCCI
    return total_cost
def BMPCost(Dimensions):
    LocFactor = 0.95386
    ENRCCI = 1.5012
    WQV = 0.
    for i in range(3):
        WQV += Dimensions[i] * 2.47105 * Dimensions[i + 3] * 3.28084    # ha to acre & m to feet

    Aimp = WQV * 12 / 0.95
    BMPcost = Aimp * 81251 * LocFactor * ENRCCI

    return (BMPcost)

def CostEstimation (values_list):

    LIDList =  values_list[:72]
 
    PondsDimensions = values_list[72:78]
    LIDPlan = LIDCombo(LIDList)  
    lid_areas = LIDAare(LIDPlan, subcatchments_data)
    LIDcost = LIDCost (lid_areas)
  
    BMPcost = BMPCost (PondsDimensions)
    totalcost = LIDcost + BMPcost
    return (totalcost)
    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.visualization.scatter import Scatter

# Define the optimization problem with constraints
class MyProblem(Problem):
    def __init__(self):

        xl = np.zeros(n)
        xu = np.full(n, 5.0)

        xu[72:75] = 0.1
        xu[75:n] = 5

        xl[72:75] = 0.04  
        xl[75:n] = 3
        
    
        # 78 continuous decision variables ranging from 0 to 10, 3 objectives, and 17 constraints
        super().__init__(n_var=n, n_obj=3, n_constr=18, xl=xl, xu=xu)

    def _evaluate(self, x, out, *args, **kwargs):
       
        UrbanPeak, RuralPeak = np.array([TotalFlow(list(xi)) for xi in x]).T
        f1 = 100 * ( RuralPeak - OTF ) / OTF
        f2 = 100 * ( UrbanPeak -  OPF ) / OPF 
       
        f3 = np.array([CostEstimation(list(xi)) for xi in x]) / 1000 

        # Define constraint groups as before
        groups = [
            [0, 1, 2, 3],               
            [4, 5, 6, 7, 8],           
            [9, 10, 11, 12],           
            [13, 14, 15, 16, 17],      
            [18, 19, 20, 21],           
            [22, 23, 24],               
            [25, 26, 27, 28],          
            [29, 30, 31, 32],           
            [33, 34, 35, 36, 37, 38],   
            [39, 40, 41, 42, 43],      
            [44, 45, 46, 47],          
            [48, 49, 50],               
            [51, 52, 53, 54, 55, 56],   
            [57, 58, 59, 60],           
            [61, 62, 63],               
            [64, 65, 66, 67],           
            [68, 69, 70, 71]            
        ]

        # Build the constraint array g, where each constraint is: sum(x[group]) - 20 <= 0
        g = np.zeros((x.shape[0], len(groups)))
        for i, group in enumerate(groups):
            g[:, i] = np.sum(x[:, group], axis=1) - 10

        g_f3 = f3 - MAXCost

        # Combine all objectives into a single array with 3 columns
        out["F"] = np.column_stack([f1, f2, f3])
        out["G"] = np.column_stack([g, g_f3])

# Define the optimization algorithm (NSGA-II)
algorithm = NSGA2(pop_size=30)

# Solve the problem
problem = MyProblem()
res = minimize(problem, algorithm, termination=get_termination("n_gen", 5000), verbose=True)

# Check and plot results
print("Result F:", res.F)

# Check results and plot in 3D
if res.F is not None and len(res.F) > 0:
    # Create a 3D scatter plot of the Pareto front
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # Extract objectives f1, f2, and f3 from the result
    f1_vals = res.F[:, 0]
    f2_vals = res.F[:, 1]
    f3_vals = res.F[:, 2]
    
    scatter = ax.scatter(f1_vals, f2_vals, f3_vals, c='blue', label="Pareto Front", marker='o')
    
    ax.set_xlabel("Peak Flow Reduction (%)")
    ax.set_ylabel("Flow Volume Reduction (%)")
    ax.set_zlabel("Cost ($1000)")
    ax.legend()
    plt.title("")
    plt.show()
else:
    print("No feasible solutions found. Try increasing population size or generations.")
